In [1]:
import numpy as np 
import pandas as pd 
import scipy.stats as stats

# DISPLAY SETTINGS 
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 500) 

# loading the data
data = pd.read_csv("eop_indexes", index_col='Id') 
data.head()  

,Age,Gender,Occupation,Marital_Status,Education,Social_habits,Present_Illness_Comorbid_Conditions,Chronic_Comorbid_Conditions,Vitals,No_Of_Medicine_Per_Day,Symptoms,Severity_Of_Symptoms,Health_Satisfaction,Affordability,Need_For_Assistance,Need_For_Medical_Treatment,Ability_To_Perform_Daily_Tasks,Task_Walking,Task_Dressing,Task_Bathing,Task_Toiletting,Task_Eating,Activity_Managing_Finances,Activity_Driving,Activity_Shopping,Activity_House_Maintenance,Activity_Food_Preparation,Activity_Managing_Communication,Activity_Taking_Medication,Access_To_Healthcare_Satisfaction,Ability_To_Concentrate_On_Daily_Activities,Energy_To_Perform_Daily_Activities,Anxiety_Hallucinations_Depression,Sleep_Satisfaction,Poor_Eye_Sight,OTC_Herbal_Medication,Adherence_To_Prescription,Relatives_With_History_Of_Hypertension_Diabetes,Hospitalization_Over_Last_Year,Complication_Due_To_Hypertension,Hypertension_Complications,Complications_Due_To_Diabetes,Diabetes_Complications,Lack_Of_Apetite_Nausea_Satiety,Symptoms2,Health_Check_Up,Control_Of_Bowel_Bladder_Movements,Precautions_Adherence_To_Selfcare,Adverse_Effects_Of_Medication,Adverse_Effects,Medication_Causing_Adverse_Effects,Medication_Dosage,Quality_Of_Life_Effected,CCI,CPS,SYMPTOM_BURDEN,TEN_YEAR_SURVIVAL_PERCENTAGE
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,68,F,None,Married,None,none,Hypertension,Hypertension,Temp-N-B.p- 130/90mmhg-RBS- 90mg/dl-Pulse rate...,6,"Dizziness, headache, nausea, back pain, chest ...",severe,no,yes,yes,yes,yes,independent,independent,independent,independent,independent,independent,independent,independent,independent,independent,independent,independent,yes,no,yes,no,no,yes,no,yes,no,yes,yes,"Stroke, stent.",no,None,yes,dizziness(on sitting or standing ) | headache ...,never,no,yes,no,None,None,None,yes,3,7,Higher,77.48
2,67,M,Employed,Married,None,"alcohol, smoking",type II DM,Diabetes Mellitus,Temp- N -B.p- 110/70mmhg-Pulse rate- 82 b/min-...,8,"Pain in right elbow and leg , swelling of righ...",severe,no,no,yes,yes,no,dependent,dependent,dependent,dependent,dependent,dependent,dependent,dependent,dependent,dependent,dependent,dependent,yes,no,no,yes,no,yes,no,yes,yes,yes,yes,"Liver cirrhosis, portal hypertension , hepatom...",yes,Diabetic neuropathy,yes,dizziness(on sitting or standing ) | pedal ede...,never,yes,yes,no,None,None,None,yes,3,9,Higher,77.48
3,67,M,Employed,Married,High school,smoking,Stroke,Stroke,temp-N-B.p-170/90mmHg-pulse rate-89 b/min-RBS-...,6,Pedal edema,severe,no,yes,yes,yes,yes,require assistance,independent,independent,independent,independent,dependent,independent,dependent,dependent,dependent,dependent,independent,yes,no,yes,no,no,yes,no,yes,no,yes,no,None,yes,Diabetic foot,yes,pedal edema | frequent thirst | frequent uri...,monthly,no,yes,no,None,None,None,yes,3,7,Higher,77.48
4,68,M,None,Married,Graduate,"alcohol, chewing tobacco",hypertension with type II DM,"Hypertension,Diabetes Mellitus",Temp-N -B.P- 140/90 mmhg-pulse rate - 74b/min-...,7,"Dizziness, cough, headache, numbness , tinglin...",severe,no,yes,yes,yes,yes,independent,independent,independent,independent,independent,independent,dependent,dependent,dependent,dependent,dependent,independent,yes,no,yes,no,no,yes,yes,yes,yes,yes,yes,"Stroke , blood clot in brain",no,None,yes,dizziness(on sitting or standing ) | headache ...,monthly,yes,yes,no,None,None,None,yes,4,9,Higher,53.39
5,72,M,Employed,Married,None,"alcohol, smoking",hypertension,Hypertension,Temp- N -B.P- 130/90mmhg -Pulse rate - 78b/min...,6,"cataract, pain in left hand , numbness and tin...",severe,no,no,yes,yes,yes,independent,independent,independent,independent,independent,dependent,dependent,dependent,dependent,dependent,dependent,independent,yes,no,no,no,yes,yes,no,yes,no,no,yes,"Stroke , paralysis",no,None,yes,dizziness(on sitting or standing ) | headache ...,monthly,no,no,no,None,None,None,yes,4,7,Higher,53.39


In [2]:
# Label encoding features
new_labels = {"SYMPTOM_BURDEN" : {'Lower':1, 'Higher':0}, 'Gender' : {'F': 0, 'M': 1 }} 

# replacing categorical values with encoded values 
data.replace(new_labels, inplace=True)
data["Quality_Of_Life_Effected"] = np.where(data["Quality_Of_Life_Effected"].str.contains("yes"), 1, 0)

In [13]:
ROP_LABELS = [0, 1, 2]
ROP_BINS = [0, 5, 8, 20]
data['ROP'] = pd.cut(data['No_Of_Medicine_Per_Day'], bins=ROP_BINS, labels=ROP_LABELS)
data.ROP.value_counts() 

1    233
0     50
2     32
Name: ROP, dtype: int64

In [14]:
# SELECTING CALCULATED FEATURES FOR MODELLING 
features = data[['Age', 'Gender', 'CCI', 'CPS', 'ROP', 'SYMPTOM_BURDEN', 'TEN_YEAR_SURVIVAL_PERCENTAGE', 'Quality_Of_Life_Effected']]
features.head() 

,Age,Gender,CCI,CPS,ROP,SYMPTOM_BURDEN,TEN_YEAR_SURVIVAL_PERCENTAGE,Quality_Of_Life_Effected
Id,,,,,,,,
1,68,0,3,7,1,0,77.48,1
2,67,1,3,9,1,0,77.48,1
3,67,1,3,7,1,0,77.48,1
4,68,1,4,9,1,0,53.39,1
5,72,1,4,7,1,0,53.39,1


In [15]:
# Libraries for predictive analysis 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier

X = features.drop('Quality_Of_Life_Effected', axis=1, inplace=False) 
y = features.Quality_Of_Life_Effected 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)
print(X_train.shape, X_test.shape) 

(236, 7) (79, 7)


In [16]:
log_reg = LogisticRegression() 
log_reg.fit(X_train, y_train) 
prediction = log_reg.predict(X_test) 
log_reg.score(X_test, y_test)

C:\Users\SiddarthaBoppana\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8481012658227848

In [18]:
features.groupby(['Gender', 'ROP']).agg([np.mean, np.median, np.count_nonzero, np.std]).Quality_Of_Life_Effected

mean  median  count_nonzero       std
Gender ROP                                           
0      0    0.406250       0             13  0.498991
       1    0.121212       0             12  0.328035
       2    0.153846       0              2  0.375534
1      0    0.222222       0              4  0.427793
       1    0.097015       0             13  0.297089
       2    0.105263       0              2  0.315302

In [6]:
from sklearn.metrics import accuracy_score, classification_report 
accuracy_score(prediction, y_test) 

0.8481012658227848

In [7]:
print(classification_report(prediction, y_test))  

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        79
           1       0.00      0.00      0.00         0

    accuracy                           0.85        79
   macro avg       0.50      0.42      0.46        79
weighted avg       1.00      0.85      0.92        79



C:\Users\SiddarthaBoppana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
import statsmodels.api as sm 
from statsmodels.formula.api import ols 


mod = ols('Age ~ ROP', data=features[features.Gender == 0]).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print('ANOVA table for Female')
print('----------------------')
print(aov_table)
print('------------------------------------------------------------------')
mod = ols('Age ~ ROP', data=features[features.Gender == 1]).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print('ANOVA table for Male')
print('----------------------')
print(aov_table)

ANOVA table for Female
----------------------
               sum_sq     df         F   PR(>F)
ROP        105.795697    2.0  4.330861  0.01495
Residual  1722.197358  141.0       NaN      NaN
------------------------------------------------------------------
ANOVA table for Male
----------------------
               sum_sq     df         F    PR(>F)
ROP         83.877455    2.0  2.127248  0.122364
Residual  3312.122545  168.0       NaN       NaN


In [27]:
# ANOVA FOR QoL
mod = ols('Age ~ ROP', data=features[features.Quality_Of_Life_Effected == 0]).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print('ANOVA table for QoL Effected (NO)')
print('----------------------')
print(aov_table)
print('------------------------------------------------------------------')
mod = ols('Age ~ ROP', data=features[features.Quality_Of_Life_Effected == 1]).fit()
aov_table = sm.stats.anova_lm(mod, typ=2)
print('ANOVA table for QoL Effected (YES)')
print('----------------------')
print(aov_table)

ANOVA table for QoL Effected (NO)
----------------------
               sum_sq     df         F    PR(>F)
ROP         85.246243    2.0  2.470641  0.086469
Residual  4588.991675  266.0       NaN       NaN
------------------------------------------------------------------
ANOVA table for QoL Effected (YES)
----------------------
              sum_sq    df         F    PR(>F)
ROP       103.311995   2.0  3.739455  0.031821
Residual  593.992353  43.0       NaN       NaN
